# This Python Script it's a draft of the functions.

In [1]:
# Importing packages and making the request to the dota buff URL
import pandas as pd
import os
from bs4 import BeautifulSoup
import re, requests
import time
import numpy as np
dota_url='https://www.dotabuff.com/heroes/meta'
r   = requests.get(dota_url, headers={'user-agent': 'Mozilla/5.0'})
r

C:\Users\User\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


<Response [200]>

In [2]:
# Creating our soup with the content
soup = BeautifulSoup(r.content)

<img src='dotabuff.png' width="800" height="800">



### Since this page it's basically a huge table, we just need to access the class table in the HTML

In [3]:
#Creating our soup table
table = soup('table')

In [4]:
#Transforming the table into a data frame and deleting the column 'Hero'
hero_df = pd.read_html(str(table))[0]
del hero_df['Hero']
hero_df

,Hero.1,Pick %,Win %,Pick %.1,Win %.1,Pick %.2,Win %.2,Pick %.3,Win %.3,Pick %.4,Win %.4
0,Anti-Mage,14.62%,48.88%,14.15%,49.11%,13.70%,50.76%,13.41%,50.54%,12.13%,50.48%
1,Axe,11.34%,51.75%,10.18%,49.54%,9.06%,49.50%,7.91%,48.06%,5.43%,47.39%
2,Bane,5.30%,48.96%,6.89%,50.98%,8.55%,52.22%,10.50%,53.06%,16.54%,54.09%
3,Bloodseeker,5.58%,51.13%,4.12%,51.89%,3.92%,51.38%,3.45%,49.74%,3.72%,49.97%
4,Crystal Maiden,11.48%,50.25%,9.74%,48.99%,8.34%,48.10%,7.19%,47.16%,5.12%,46.72%
...,...,...,...,...,...,...,...,...,...,...,...
117,Void Spirit,8.52%,47.25%,9.69%,47.86%,11.65%,49.96%,13.29%,49.91%,15.66%,49.06%
118,Snapfire,8.73%,48.21%,10.48%,48.17%,11.56%,47.31%,12.16%,46.87%,15.07%,46.90%
119,Mars,7.35%,44.32%,10.46%,45.38%,12.51%,46.66%,14.28%,46.78%,19.17%,46.88%
120,Dawnbreaker,4.78%,50.16%,5.99%,49.58%,6.29%,49.60%,6.81%,50.22%,7.44%,48.29%


In [5]:
#Rename columns
hero_df.columns = ['Hero', 'Pick%_Crusader', 'Win%_Crusader', 'Pick%_Crusader_Archon','Win%_Crusader_Archon',
                  'Pick%_Legend','Win%_Legend','Pick%Ancient','Win%Ancient','Pick%Divine_Imortal','Win%Divine_Imortal']

In [6]:
#all of our columns has string as values, we have to remove the '%' character and then transform it to numerical value
#Let's create a for looping to access all of our columns that should had numerical values, remove the '%' and transform it to numerical
for y in hero_df.columns:
    if y !='Hero':
        for w in range(len(hero_df[y])):
            hero_df[y][w] = hero_df[y][w].translate({ord(i):None for i in '%'})
            hero_df[y][w]=pd.to_numeric(hero_df[y][w])
            
        

In [7]:
#Ok, now that all of our columns are numerical, let's give our index column a name
hero_df.reset_index(inplace=True)

In [8]:
#Now we need to create  our table with the info about which heroes are best against and worst against
#Since we are dealing with 122 heroes, i would need 122 html to get the info
# this is a tedious task to get one by one. So first we need to create a list with all of our heroes:
#We will create in a manner that, even if the heroes meta change, we still be able to use this script
#Creating the list...
hero_name = hero_df['Hero']
hero_name
#If we take a look, the html for each hero is like that: <https://www.dotabuff.com/heroes/HERO-NAME>
# letters should be all in lower case and instead of space between the name it should be "-". Let's change our hero_name object:

hero_name_new = []
for i in range(len(hero_name)):
    if "-" in hero_name[i]:
        hero_name_new.append(hero_name[i].lower())
    elif "'" in hero_name[i]:
        hero_name_new.append(hero_name[i].lower().replace("'","").replace(' ','-'))
    else:
        hero_name_new.append(hero_name[i].lower().replace(' ',"-"))

In [9]:
#Now we can create several soups for our heroes
#First, we are going to create a list of urls:
heroes_url = []
for i in range(len(hero_name_new)):
    heroes_url.append('https://www.dotabuff.com/heroes/'+hero_name_new[i])
(heroes_url)

['https://www.dotabuff.com/heroes/anti-mage',
 'https://www.dotabuff.com/heroes/axe',
 'https://www.dotabuff.com/heroes/bane',
 'https://www.dotabuff.com/heroes/bloodseeker',
 'https://www.dotabuff.com/heroes/crystal-maiden',
 'https://www.dotabuff.com/heroes/drow-ranger',
 'https://www.dotabuff.com/heroes/earthshaker',
 'https://www.dotabuff.com/heroes/juggernaut',
 'https://www.dotabuff.com/heroes/mirana',
 'https://www.dotabuff.com/heroes/morphling',
 'https://www.dotabuff.com/heroes/shadow-fiend',
 'https://www.dotabuff.com/heroes/phantom-lancer',
 'https://www.dotabuff.com/heroes/puck',
 'https://www.dotabuff.com/heroes/pudge',
 'https://www.dotabuff.com/heroes/razor',
 'https://www.dotabuff.com/heroes/sand-king',
 'https://www.dotabuff.com/heroes/storm-spirit',
 'https://www.dotabuff.com/heroes/sven',
 'https://www.dotabuff.com/heroes/tiny',
 'https://www.dotabuff.com/heroes/vengeful-spirit',
 'https://www.dotabuff.com/heroes/windranger',
 'https://www.dotabuff.com/heroes/zeus',


In [10]:
#Now let's create our requests and our soups. We are going to use time.sleep function, since we are not wanting to do a ddos attack:
requests_hero   = []

for i in range(len(heroes_url)):
    time.sleep(0.3)
    requests_hero.append(requests.get(heroes_url[i], headers={'user-agent': 'Mozilla/5.0'}))
requests_hero #Checking if all of our response are [200]

122

In [18]:
#Ok, now let's create our soups!
soups_heroes =[] 
for i in range(len(requests_hero)):
    soups_heroes.append(BeautifulSoup(requests_hero[i].content))




In [23]:
#Creating our first data frame. It will be about the heroes advantage in game:
#The advantage table in each URL are in position 3, so we need to access table[3] of every URL. Let's make a for looping!
#Inside this for looping we will also create, delete and rename some columns. We also are going to transform into a numerical value some columns
list_advantage=[]
for i in range(len(soups_heroes)):
    
    
    a = pd.read_html(str(soups_heroes[i]('table')))[3]
    a['MetaHero'] = hero_df['Hero'][i]
    del a['Hero']
    a.rename(columns = {'Hero.1':'Better against'},inplace = True)
    a['MetaHero'] = hero_df['Hero'][i]
    list_advantage.append(a)
    df_advantage = pd.concat(list_advantage)
    df_advantage.reset_index(drop=True,inplace=True)
    for w in df_advantage.columns:
        if ((w!='Better against') and(w!='Matches') and(w!='MetaHero')):
            for z in range(len(df_advantage.index)):
                s =  df_advantage.at[z,w].replace('%','')
                df_advantage.at[z,w] = pd.to_numeric(s)


ValueError: No tables found

In [76]:
#Following the same logic, we will create a data frame for the disadvantage. It will be the table 4 in each URL
list_disadvantage=[]
for i in range(len(soups_heroes)):
    
    a = pd.read_html(str(soups_heroes[i]('table')))[4]
    a['MetaHero'] = hero_df['Hero'][i]
    del a['Hero']
    a.rename(columns = {'Hero.1':'Worst against'},inplace = True)
    
    list_disadvantage.append(a)
    df_disadvantage = pd.concat(list_disadvantage)
    df_disadvantage.reset_index(drop=True,inplace=True)
    for w in df_disadvantage.columns:
        if ((w!='Worst against') and(w!='Matches') and(w!='MetaHero')):
            for z in range(len(df_disadvantage.index)):
                s =  df_disadvantage.at[z,w].replace('%','')
                df_disadvantage.at[z,w] = pd.to_numeric(s)

In [77]:
#For lane position, it will be table 1
list_lane_position=[]
for i in range(len(soups_heroes)):
    a = pd.read_html(str(soups_heroes[i]('table')))[1]
    a['MetaHero'] = hero_df['Hero'][i]    
    list_lane_position.append(a)
    df_lane_position = pd.concat(list_lane_position)
    df_lane_position.reset_index(drop=True,inplace=True)
    for w in df_lane_position.columns:
        if ((w!='Lane')and(w!='KDA Ratio')and(w!='GPM')and(w!='XPM')and(w!='MetaHero')):
            for z in range(len(df_lane_position.index)):
                s =  df_lane_position.at[z,w].replace('%','')
                df_lane_position.at[z,w] = pd.to_numeric(s)

In [78]:
# For the items, it will be table 2
list_item_hero=[]
for i in range(len(soups_heroes)):
    a = pd.read_html(str(soups_heroes[i]('table')))[2]
    a['MetaHero'] = hero_df['Hero'][i]
    del a['Item']
    a.rename(columns = {'Item.1':'Item'},inplace = True)
    list_item_hero.append(a)
    df_item_hero = pd.concat(list_item_hero)
    df_item_hero.reset_index(drop=True,inplace=True)
    for w in df_item_hero.columns:
        if ((w!='Item')and(w!='Matches')and(w!='Wins')and(w!='MetaHero')):
            for z in range(len(df_item_hero.index)):
                s =  df_item_hero.at[z,w].replace('%','')
                df_item_hero.at[z,w] = pd.to_numeric(s)

In [79]:
#Now we are going to create our first function!
# We will create a function that access our first dataframe about the heroes that are in the meta game
#Basically, it's several if else to return the info
def get_top5(by,elo,self):
    if ((by =='Pick')and(elo=='Crusader')):
        return self.hero_df.groupby(['Pick%_Crusader','Hero']).sum().sort_values('Pick%_Crusader',ascending=False).head(5)
    
    elif((by=='Pick')and(elo=='Crusader_Archon')):
        return self.hero_df.groupby(['Pick%_Crusader_Archon','Hero']).sum().sort_values('Pick%_Crusader_Archon',ascending=False).head(5)
    
    elif((by=='Pick')and(elo=='Legend')):
        return self.hero_df.groupby(['Pick%_Legend','Hero']).sum().sort_values('Pick%_Legend',ascending=False).head(5)
    
    elif((by=='Pick')and (elo=='Ancient')):
        return self.hero_df.groupby(['Pick%Ancient','Hero']).sum().sort_values('Pick%Ancient',ascending=False).head(5)
    
    elif((by=='Pick')and(elo=='Divine_Imortal')):
        return self.hero_df.groupby(['Pick%Divine_Imortal','Hero']).sum().sort_values('Pick%Divine_Imortal',ascending=False).head(5)
    
    elif((by=='Win')and(elo=='Crusader')):
        return self.hero_df.groupby(['Win%_Crusader','Hero']).sum().sort_values('Win%_Crusader',ascending=False).head(5)
    
    elif((by=='Win')and(elo=='Crusader_Archon')):
        return self.hero_df.groupby(['Win%_Crusader_Archon','Hero']).sum().sort_values('Win%_Crusader_Archon',ascending=False).head(5)
    
    elif((by=='Win')and(elo=='Legend')):
        return self.hero_df.groupby(['Win%_Legend','Hero']).sum().sort_values('Win%_Legend',ascending=False).head(5)
    
    elif((by=='Win')and(elo=='Ancient')):
        return self.hero_df.groupby(['Win%Ancient','Hero']).sum().sort_values('Win%Ancient',ascending=False).head(5)
    
    elif((by=='Win')and(elo=='Divine_Imortal')):
        return self.hero_df.groupby(['Win%Divine_Imortal','Hero']).sum().sort_values('Win%Divine_Imortal',ascending=False).head(5)

In [83]:
#For our second function we will make a join between both df_advantage and df_disadvantage
def get_advantage_disadvantage(hero,hero_df,df_advantage,df_disadvantage):
    my_list=[]
    for i in hero_df['Hero']:
        
        for w in hero:
            
            if w ==i:
                for w in hero:
                    a=df_advantage[df_advantage['MetaHero']==w].join(df_disadvantage[df_disadvantage['MetaHero']==w],
                                                                     how='left',lsuffix='_Advantage',rsuffix='_Disadvantage')
                    my_list.append(a)
                    my_df=pd.concat(my_list)
                return my_df
                

In [93]:
# For the get_lane_position and get_item_hero it will be a simple groupby to access the information:
def get_lane_position(self,by):
    my_list=[]
    for i in self.df_lane_position['MetaHero'].unique():
        for w in by:
            if w==i:
                a=self.df_lane_position[self.df_lane_position['MetaHero']==w].groupby(['Presence','Win Rate','KDA Ratio','GPM','XPM','MetaHero']).sum().sort_values('Win Rate',ascending=False)
                my_list.append(a)

                my_df=pd.concat(my_list)
    return my_df

In [85]:
def get_item_hero(hero,hero_df,df_item_hero):
    my_list=[]
    for i in hero_df['Hero']:
        for w in hero:
            if w==i:
                a = df_item_hero[df_item_hero['MetaHero']==w].groupby(['Win Rate','Item','MetaHero']).sum().sort_values('Win Rate',ascending=False)
                my_list.append(a)
                my_df=pd.concat(my_list)
    return my_df

In [178]:
#For our last  function, this one it's a little bit different.
#First, we need to get the URL for each item the user wants to get info
#Then, we will access this URL and scrape the info about attribute of the item and also the description
def item_info(self,item):
    
    item_name=[]
    for i in range(len(item)):
        
        if "'" in item[i]:
            item_name.append(item[i].lower().replace("'","").replace(' ','-'))
        else:
            item_name.append(item[i].lower().replace(' ','-'))
        
    items_url = []
    for i in range(len(item_name)):
        items_url.append('https://www.dotabuff.com/items/'+item_name[i])
       
    
    requests_item   = []

    for i in range(len(items_url)):
        
        requests_item.append(requests.get(items_url[i], headers={'user-agent': 'Mozilla/5.0'}))
    
    soups_items =[] 
    for i in range(len(requests_item)):
        soups_items.append(BeautifulSoup(requests_item[i].content))
        
    for w in range(len(soups_items)):
        print('\033[1m' + soups_items[w].find("div",class_='name').text + '\033[0m','\n',soups_items[w].find("div",class_='stats').text)
        print(soups_items[w].find("div",class_='description').text)

